## Import important libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests

### In order to make an API request we need API key and Channel id

In [5]:
# API_key for Youtube API
API_KEY = '***********************' ## Confidential...Not to be shared
CHANNEL_ID = 'UCoaKEeilUC7r91S5-D5gsjQ'

### Testing GitHub API

In [9]:
# Test API
## We need to pass the url location for that API and the type of file we want to get data as.
response = requests.get('http://api.github.com').json()

In [15]:
response['emojis_url'] 
## This is a dictonary output and we can use keys to call values

'https://api.github.com/emojis'

In [17]:
response.keys()

dict_keys(['current_user_url', 'current_user_authorizations_html_url', 'authorizations_url', 'code_search_url', 'commit_search_url', 'emails_url', 'emojis_url', 'events_url', 'feeds_url', 'followers_url', 'following_url', 'gists_url', 'hub_url', 'issue_search_url', 'issues_url', 'keys_url', 'label_search_url', 'notifications_url', 'organization_url', 'organization_repositories_url', 'organization_teams_url', 'public_gists_url', 'rate_limit_url', 'repository_url', 'repository_search_url', 'current_user_repositories_url', 'starred_url', 'starred_gists_url', 'topic_search_url', 'user_url', 'user_organizations_url', 'user_repositories_url', 'user_search_url'])

## Making 1st Youtube API call using various parameters.
### These parameters are given in the YouTube API documentation. We pick what's needed.

In [18]:
## For Youtube API we need to pick different parameters according to our need...
pageToken = ""
url = 'https://www.googleapis.com/youtube/v3/search?key='+ API_KEY +'&channelID='+ CHANNEL_ID+'&part=snippet'+pageToken

youtube_request = requests.get(url).json()
## This will include basic channel information in the form of dictionaries...

In [21]:
youtube_request['items'][0]
## This is in the form of nested list dictionary...

{'kind': 'youtube#searchResult',
 'etag': 'wqxqqlG4IHUOsnPiVzQIIOCUw0E',
 'id': {'kind': 'youtube#video', 'videoId': 'F2yswKCPGoI'},
 'snippet': {'publishedAt': '2021-12-01T03:09:08Z',
  'channelId': 'UCa6vGFO9ty8v5KZJXQxdhaw',
  'title': 'Jimmy Kimmel Has Had Enough of Scumbags Attacking Dr. Fauci for Trying to Keep Us Alive',
  'description': 'There is a nationwide shortage of maple syrup in Canada forcing them to withdraw from their strategic maple reserves, the CDC is recommending that every ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/F2yswKCPGoI/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/F2yswKCPGoI/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/F2yswKCPGoI/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Jimmy Kimmel Live',
  'liveBroadcastContent': 'none',
  'publishTime': '2021-12-01T03:09:08Z'}}

### Extracting required video elements from the JSON file.

In [23]:
video_id = youtube_request['items'][0]['id']['videoId']

In [25]:
video_title = youtube_request['items'][0]['snippet']['title']

In [30]:
upload_date = youtube_request['items'][0]['snippet']['publishedAt'][:10]
## We can also use split method and save date on the left of split

### We create an empty dataframe to store our derived elements

In [33]:
df = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count',
                          'dislike_count','comment_count'])

## Create a for loop to extract values for every video and include the 2nd API call in it.
### The 2nd API call has to be passed after the extraction od video_id as it will be used here.
### Lastly we use .append() method to fill our dataframe with the extracted values.

In [36]:
## To collect all this information for all the videos we create a for loop.
for video in youtube_request['items']:
    if video['id']['kind'] == 'youtube#video':
        video_id = video['id']['videoId']
        video_title = video['snippet']['title']
        upload_date = video['snippet']['publishedAt'][:10]
        
        ## 2nd API request to collect all the required stats
        url_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
        stats_request = requests.get(url_stats).json()
        
        view_count = stats_request['items'][0]['statistics']['viewCount']
        like_count = stats_request['items'][0]['statistics']['likeCount']
        dislike_count = stats_request['items'][0]['statistics']['dislikeCount']
        comment_count = stats_request['items'][0]['statistics']['commentCount']
        
        
        ## After running API request we store the data in the dataframe
        ## We pass the a dictionary with column name as key and corresponding values
        df = df.append({'video_id': video_id, 'video_title': video_title, 'upload_date': upload_date,
                       'view_count': view_count, 'like_count': like_count, 'dislike_count': dislike_count,
                       'comment_count': comment_count}, ignore_index= True)

In [38]:
df

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,F2yswKCPGoI,Jimmy Kimmel Has Had Enough of Scumbags Attack...,2021-12-01,1197109,35000,2309,4126
1,6ilZeMFyNE8,LN+ EN VIVO las 24 horas,2021-11-01,9714535,25110,3463,0


## This way we can extract data from the YouTube API.